<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/3_3_4___data_analyst_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -q langchain langchain-openai langchain_experimental

Komenda `!pip install` służy do zainstalowania niezbędnych bibliotek Pythona:

- `langchain` to framework do budowania aplikacji opartych na modelach językowych. Pozwala na tworzenie łańcuchów przetwarzania tekstu i łączenie różnych komponentów
- `langchain-openai` to integracja LangChain z API OpenAI
- `langchain_experimental` zawiera eksperymentalne funkcje LangChain, które nie są jeszcze gotowe do włączenia do głównej biblioteki

Flaga `-q` (quiet) sprawia, że pip wyświetla tylko najważniejsze komunikaty podczas instalacji, pomijając szczegółowe logi.

Instalowanie odbywa się bezpośrednio w notebooku Jupyter dzięki wykrzyknikowi (`!`) na początku linii - jest to sposób na wykonywanie poleceń systemowych z poziomu komórki Jupytera.

In [ ]:
import os
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents import AgentType
from langchain_openai import ChatOpenAI
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from google.colab import userdata


Ten kod importuje niezbędne biblioteki i moduły:

`os` - daje dostęp do funkcji systemu operacyjnego, np. do operacji na plikach i katalogach

`create_pandas_dataframe_agent` z `langchain_experimental.agents.agent_toolkits` - tworzy agenta, który potrafi pracować z ramkami danych Pandas

`AgentType` z `langchain.agents` - definiuje różne typy agentów dostępne w LangChain

`ChatOpenAI` z `langchain_openai` - interfejs do modelu GPT od OpenAI w trybie czatu

`pandas` jako `pd` - biblioteka do analizy danych, importowana pod standardowym aliasem pd

`numpy` jako `np` - biblioteka do obliczeń numerycznych, zaimportowana pod typowym aliasem np

`datetime` i `timedelta` z modułu `datetime` - służą do operacji na datach i czasie

`userdata` z `google.colab` - moduł do zarządzania danymi użytkownika w środowisku Google Colab

In [ ]:
class CFG:
    model = 'gpt-4o-mini'
    temp = 0.3

In [ ]:
api_key = userdata.get('openaivision')
os.environ['OPENAI_API_KEY'] = api_key

Te dwie linie służą do konfiguracji klucza API dla OpenAI:

`api_key = userdata.get('openaivision')` - pobiera klucz API zapisany w zmiennej 'openaivision' w danych użytkownika Google Colab. To bezpieczniejszy sposób przechowywania kluczy API niż zapisywanie ich bezpośrednio w kodzie.

`os.environ['OPENAI_API_KEY'] = api_key` - zapisuje pobrany klucz API jako zmienną środowiskową o nazwie 'OPENAI_API_KEY'. Biblioteka OpenAI automatycznie szuka klucza API właśnie w tej zmiennej środowiskowej, dzięki czemu będzie mogła nawiązać połączenie z API OpenAI.

# Functions

In [ ]:
def create_data(nrows = 300):

  # Generate sample data
  n_rows = 1000

  # Generate dates
  start_date = datetime(2024, 1, 1)
  dates = [start_date + timedelta(days=i) for i in range(nrows)]

  # Define data categories
  makes = ['Toyota', 'Honda', 'Ford', 'Chevrolet', 'Nissan', 'BMW', 'Mercedes', 'Audi', 'Hyundai', 'Kia']
  models = ['Sedan', 'SUV', 'Truck', 'Hatchback', 'Coupe', 'Van']
  colors = ['Red', 'Blue', 'Black', 'White', 'Silver', 'Gray', 'Green']

  # Create the dataset
  data = {
      'Date': dates,
      'Make': np.random.choice(makes, nrows),
      'Model': np.random.choice(models, nrows),
      'Color': np.random.choice(colors, nrows),
      'Year': np.random.randint(2015, 2023, nrows),
      'Price': np.random.uniform(20000, 80000, nrows).round(2),
      'Mileage': np.random.uniform(0, 100000, nrows).round(0),
      'EngineSize': np.random.choice([1.6, 2.0, 2.5, 3.0, 3.5, 4.0], nrows),
      'FuelEfficiency': np.random.uniform(20, 40, nrows).round(1),
      'SalesPerson': np.random.choice(['Alice', 'Bob', 'Charlie', 'David', 'Eva'], nrows)
  }

  # Create DataFrame and sort by date
  df = pd.DataFrame(data).sort_values('Date')

  return df



Ta funkcja `create_data` generuje przykładowe dane sprzedaży samochodów:

`def create_data(nrows = 300):` - funkcja przyjmuje parametr określający liczbę wierszy (domyślnie 300)

Generowane są daty, zaczynając od 1 stycznia 2024:
- `start_date = datetime(2024, 1, 1)`
- `dates = [start_date + timedelta(days=i) for i in range(nrows)]`

Zdefiniowane są listy możliwych wartości:
- `makes` - marki samochodów (Toyota, Honda, Ford itd.)
- `models` - typy nadwozia (Sedan, SUV, Truck itd.)
- `colors` - kolory pojazdów
- Lista sprzedawców: Alice, Bob, Charlie, David, Eva

Tworzony jest słownik `data` zawierający:
- Daty transakcji
- Losowo wybrane marki, modele i kolory
- Rok produkcji (losowy między 2015-2023)
- Cena (losowa między 20000-80000)
- Przebieg (losowy między 0-100000)
- Pojemność silnika (losowa z listy: 1.6, 2.0, 2.5, 3.0, 3.5, 4.0)
- Spalanie (losowe między 20-40)
- Przypisany losowo sprzedawca

Na końcu dane są przekształcane w DataFrame i sortowane po dacie.

# Data

In [ ]:
df = create_data(100)

In [ ]:
print(df.head())

        Date       Make      Model  Color  Year     Price  Mileage  \
0 2024-01-01        BMW      Coupe  White  2016  25267.22  87179.0   
1 2024-01-02   Mercedes      Sedan  White  2021  36122.85  44684.0   
2 2024-01-03    Hyundai        Van  White  2017  64826.22  91098.0   
3 2024-01-04  Chevrolet  Hatchback  Green  2021  67401.42  96582.0   
4 2024-01-05    Hyundai  Hatchback  White  2020  51813.96  40755.0   

   EngineSize  FuelEfficiency SalesPerson  
0         4.0            35.9     Charlie  
1         2.5            31.3     Charlie  
2         3.0            39.7         Bob  
3         4.0            37.9     Charlie  
4         2.0            30.1       Alice  


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            100 non-null    datetime64[ns]
 1   Make            100 non-null    object        
 2   Model           100 non-null    object        
 3   Color           100 non-null    object        
 4   Year            100 non-null    int64         
 5   Price           100 non-null    float64       
 6   Mileage         100 non-null    float64       
 7   EngineSize      100 non-null    float64       
 8   FuelEfficiency  100 non-null    float64       
 9   SalesPerson     100 non-null    object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 7.9+ KB


In [ ]:
print(df.describe())

                      Date         Year         Price       Mileage  \
count                  100   100.000000    100.000000    100.000000   
mean   2024-02-19 12:00:00  2018.410000  50365.468800  46873.880000   
min    2024-01-01 00:00:00  2015.000000  21615.910000    637.000000   
25%    2024-01-25 18:00:00  2016.000000  35755.697500  21925.500000   
50%    2024-02-19 12:00:00  2018.000000  49632.600000  43122.500000   
75%    2024-03-15 06:00:00  2020.250000  66469.580000  73665.750000   
max    2024-04-09 00:00:00  2022.000000  79878.450000  99866.000000   
std                    NaN     2.331579  17395.234794  29345.917436   

       EngineSize  FuelEfficiency  
count  100.000000      100.000000  
mean     2.718000       31.388000  
min      1.600000       21.000000  
25%      2.000000       27.075000  
50%      2.500000       31.450000  
75%      3.500000       35.600000  
max      4.000000       39.900000  
std      0.869387        5.175628  


# Agent

In [ ]:
muh_agent = create_pandas_dataframe_agent(
    ChatOpenAI(model =  CFG.model, temperature = CFG.temp),
    df, verbose=True,
    allow_dangerous_code = True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

Ta linia tworzy agenta do analizy danych wykorzystując LangChain i model językowy:

`create_pandas_dataframe_agent` tworzy specjalnego agenta, który potrafi pracować z DataFrame'ami Pandas.

Parametry:
- `ChatOpenAI(model = CFG.model, temperature = CFG.temp)` - konfiguracja modelu językowego:
  - używa modelu określonego w zmiennej CFG.model
  - temperature (losowość odpowiedzi) jest ustawiona zgodnie z CFG.temp

- `df` - DataFrame z danymi, który agent będzie analizował

- `verbose=True` - włącza szczegółowe logowanie działań agenta

- `allow_dangerous_code = True` - pozwala agentowi na wykonywanie potencjalnie niebezpiecznych operacji

- `agent_type=AgentType.OPENAI_FUNCTIONS` - używa agenta typu OPENAI_FUNCTIONS, który wykorzystuje funkcje API OpenAI do analizy danych

Stworzony agent `muh_agent` będzie mógł odpowiadać na pytania dotyczące danych i wykonywać na nich operacje.

In [ ]:
def ask_agent(agent, question):
  response = agent.run({
        "input": question,
        "agent_scratchpad": f"Human: {question}\nAI: To answer this question, I need to use Python to analyze the dataframe. \
        I'll use the python_repl_ast tool.\n\nAction: python_repl_ast\nAction Input: ",
    })
  print(f"Question: {question}")
  print(f"Answer: {response}")


Ta funkcja `ask_agent` obsługuje komunikację z agentem:

`def ask_agent(agent, question):` - przyjmuje dwa parametry: agenta i pytanie

Wywołuje agenta używając metody `run()` z następującymi parametrami:
- `"input"` - przekazane pytanie
- `"agent_scratchpad"` - zawiera:
  - pytanie od użytkownika ("Human: {question}")
  - wstępną odpowiedź AI informującą o potrzebie użycia Pythona
  - informację o użyciu narzędzia python_repl_ast

Następnie funkcja wyświetla:
- zadane pytanie (`Question: {question}`)
- otrzymaną odpowiedź (`Answer: {response}`)

Ta funkcja służy jako interfejs między użytkownikiem a agentem - pozwala zadawać pytania w naturalnym języku i otrzymywać odpowiedzi oparte na analizie danych z DataFrame.

In [ ]:
ask_agent(muh_agent, "What are the column names in this dataset?")


<ipython-input-11-2a1390171e96>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run({




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.columns.tolist()'}`


['Date', 'Make', 'Model', 'Color', 'Year', 'Price', 'Mileage', 'EngineSize', 'FuelEfficiency', 'SalesPerson']The column names in the dataset are:

1. Date
2. Make
3. Model
4. Color
5. Year
6. Price
7. Mileage
8. EngineSize
9. FuelEfficiency
10. SalesPerson

> Finished chain.
Question: What are the column names in this dataset?
Answer: The column names in the dataset are:

1. Date
2. Make
3. Model
4. Color
5. Year
6. Price
7. Mileage
8. EngineSize
9. FuelEfficiency
10. SalesPerson


In [ ]:
ask_agent(muh_agent, "How many rows are in this dataset?")




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


100The dataset contains 100 rows.

> Finished chain.
Question: How many rows are in this dataset?
Answer: The dataset contains 100 rows.


In [ ]:
ask_agent(muh_agent, "What is the average price of cars sold?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Price'].mean()"}`


50365.4688The average price of cars sold is approximately $50,365.47.

> Finished chain.
Question: What is the average price of cars sold?
Answer: The average price of cars sold is approximately $50,365.47.
